In [0]:
%run ipums_lib.py
rows = row_generator(datapath = "usa_00201.dat", ddipath = "usa_00201.xml")

In [2]:
data = []
for index, row in enumerate(rows):
    data.append(row)
print(len(data))

1881540


In [0]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline
import seaborn as sns

In [0]:
df = pd.DataFrame(data=data)

In [6]:
df.to_csv('ipums_raw.csv', index=False)
df

,YEAR,SAMPLE,SERIAL,CBSERIAL,HHWT,CLUSTER,STATEFIP,COUNTYFIP,MET2013,CITY,CITYPOP,PUMA,STRATA,GQ,MORTGAGE,MORTAMT1,HHINCOME,VALUEH,PERNUM,CBPERNUM,PERWT,NCHILD,NCHLT5,RELATE,RELATED,SEX,AGE,MARST,BIRTHYR,MARRNO,MARRINYR,YRMARR,DIVINYR,WIDINYR,FERTYR,RACE,RACED,HISPAN,HISPAND,BPL,...,EMPSTATD,LABFORCE,OCC,IND,CLASSWKR,CLASSWKRD,UHRSWORK,WRKLSTWK,INCTOT,FTOTINC,INCWAGE,POVERTY,MIGRATE1,MIGRATE1D,VETSTAT,VETSTATD,EDUCD_HEAD,EDUCD_MOM,EDUCD_POP,EDUCD_SP,EDUCD_MOM2,EDUCD_POP2,EMPSTAT_HEAD,EMPSTAT_MOM,EMPSTAT_POP,EMPSTAT_SP,EMPSTAT_MOM2,EMPSTAT_POP2,INCTOT_HEAD,INCTOT_MOM,INCTOT_POP,INCTOT_SP,INCTOT_MOM2,INCTOT_POP2,INCWAGE_HEAD,INCWAGE_MOM,INCWAGE_POP,INCWAGE_SP,INCWAGE_MOM2,INCWAGE_POP2
0,2018,201801,00000007,2018010000510,00000018.00,2018000000071,01,000,00000,0000,00000,00400,000000040001,3,0,00000,9999999,9999999,0001,01,00000018.00,0,0,13,1301,2,066,5,1952,1,1,1992,1,1,0,1,100,0,000,001,...,30,1,0000,0000,0,00,00,3,0015000,9999999,000000,000,1,10,1,11,,,,,,,,,,,,,,,,,,,,,,,,
1,2018,201801,00000011,2018010001151,00000092.00,2018000000111,01,000,00000,0000,00000,02200,000000220001,3,0,00000,9999999,9999999,0001,01,00000092.00,0,0,13,1301,1,065,2,1953,1,1,1993,1,1,0,2,200,0,000,001,...,30,1,0000,0000,0,00,00,3,0012000,9999999,000000,000,1,10,1,11,,,,,,,,,,,,,,,,,,,,,,,,
2,2018,201801,00000012,2018010001207,00000031.00,2018000000121,01,000,00000,0000,00000,02400,000000240001,3,0,00000,9999999,9999999,0001,01,00000031.00,0,0,13,1301,2,079,5,1939,1,1,1986,1,1,0,2,200,0,000,013,...,30,1,0000,0000,0,00,00,3,0009900,9999999,000000,000,1,10,1,11,,,,,,,,,,,,,,,,,,,,,,,,
3,2018,201801,00000013,2018010001284,00000016.00,2018000000131,01,073,13820,0000,00000,01303,000000130301,4,0,00000,9999999,9999999,0001,01,00000016.00,0,0,12,1270,1,057,3,1961,1,1,1988,1,1,0,7,700,4,460,260,...,30,1,0000,0000,0,00,00,1,0001400,9999999,000000,010,2,24,2,20,,,,,,,,,,,,,,,,,,,,,,,,
4,2018,201801,00000014,2018010001318,00000071.00,2018000000141,01,000,00000,0000,00000,00500,000000050001,3,0,00000,9999999,9999999,0001,01,00000071.00,0,0,13,1301,2,081,5,1937,1,1,1985,1,1,0,1,100,0,000,001,...,30,1,0000,0000,0,00,00,3,0012000,9999999,000000,000,1,10,1,11,,,,,,,,,,,,,,,,,,,,,,,,
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1881535,2018,201801,01410973,2018001399964,00000080.00,2018014109731,56,000,00000,0000,00000,00500,000000050056,1,3,01000,0077100,0275000,0001,01,00000080.00,0,0,01,0101,2,055,1,1963,3,1,2010,1,1,0,1,100,0,000,006,...,10,2,3220,8191,2,23,30,2,0045000,0077100,045000,460,1,10,1,11,081,,,063,,,1,,,3,,,0045000,,,0032100,,,045000,,,000000,,
1881536,2018,201801,01410973,2018001399964,00000080.00,2018014109731,56,000,00000,0000,00000,00500,000000050056,1,3,01000,0077100,0275000,0002,02,00000081.00,0,0,02,0201,1,077,1,1941,3,1,2010,1,1,0,1,100,0,000,029,...,30,1,3870,9470,2,27,00,1,0032100,0077100,000000,460,1,10,2,20,081,,,081,,,1,,,1,,,0045000,,,0045000,,,045000,,,045000,,
1881537,2018,201801,01410974,2018001400326,00000102.00,2018014109741,56,000,00000,0000,00000,00400,000000040056,1,1,00000,0089600,0260000,0001,01,00000102.00,3,0,01,0101,1,041,4,1977,1,1,2004,1,1,0,1,100,0,000,056,...,10,2,5000,8370,2,22,40,3,0042100,0042100,042100,329,1,10,1,11,065,,,101,,,1,,,1,,,0042100,,,0047500,,,042100,,,047500,,
1881538,2018,201801,01410975,2018001400502,00000050.00,2018014109751,56,000,00000,0000,00000,00100,000000010056,1,0,00000,0030000,9999999,0001,01,00000049.00,1,0,01,0101,2,049,4,1969,1,1,1993,1,1,1,1,100,0,000,056,...,10,2,3603,8270,2,22,40,2,0018500,0030000,018500,181,1,10,1,11,071,,,,,,1,,,,,,0018500,,,,,,018500,,,,,


In [7]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1881540 entries, 0 to 1881539
Data columns (total 94 columns):
YEAR            object
SAMPLE          object
SERIAL          object
CBSERIAL        object
HHWT            object
CLUSTER         object
STATEFIP        object
COUNTYFIP       object
MET2013         object
CITY            object
CITYPOP         object
PUMA            object
STRATA          object
GQ              object
MORTGAGE        object
MORTAMT1        object
HHINCOME        object
VALUEH          object
PERNUM          object
CBPERNUM        object
PERWT           object
NCHILD          object
NCHLT5          object
RELATE          object
RELATED         object
SEX             object
AGE             object
MARST           object
BIRTHYR         object
MARRNO          object
MARRINYR        object
YRMARR          object
DIVINYR         object
WIDINYR         object
FERTYR          object
RACE            object
RACED           object
HISPAN          object
HISPAND      

In [8]:
len(df.columns)

94

In [0]:
df = df.apply(pd.to_numeric, errors='ignore')

In [0]:
df = df[['POVERTY', 'MIGRATE1', 'VETSTAT', 
         'UHRSWORK', 'OCC', 'EMPSTAT', 'YRMARR', 
         'ANCESTR1', 'CLASSWKR', 'EDUC', 'INCTOT', 'RACE', 
         'HISPAN', 'BPL', 'DEGFIELD', 'FERTYR', 'DIVINYR',
         'WIDINYR', 'AGE', 'BIRTHYR', 'MORTGAGE', 'SEX', 'PERWT',
         'STRATA', 'CLUSTER', 'MARRNO', 'NCHILD', 'STATEFIP',
         'COUNTYFIP', 'MARST', 'NCHLT5', 'VALUEH', 'CITY',
         'HHINCOME', 'MORTAMT1', 'YEAR']]

In [0]:
df['AGEMARR'] = df['YRMARR']-df['BIRTHYR']

In [0]:
df = df.drop(columns = 'POVERTY')

In [0]:
df.RACE = df.RACE.map({1:1, 2:2, 3:3, 4:4, 5:4, 6:4, 7:5, 8:5, 9:5})

In [0]:
df.HISPAN = df.HISPAN.map({0:0, 1:1, 2:2, 3:2, 4:2, 9:3})

In [0]:
df = df[df["VETSTAT"] != 0.0]

In [23]:
df[df["MIGRATE1"] == 9].shape

(0, 37)

In [27]:
df = df[df['EMPSTAT'] != '0']

/usr/local/lib/python3.6/dist-packages/pandas/core/ops/__init__.py:1115: FutureWarning: elementwise comparison failed; returning scalar instead, but in the future will perform elementwise comparison
  result = method(y)


In [0]:
df.EDUC = df.EDUC.map({0:0, 1:0, 2:0, 3:1, 4:1, 5:1, 6:1, 7:2, 8:2, 9:2, 10:2, 11:3})

In [0]:
df = df.drop(columns = "FERTYR")

In [0]:
df = df.drop(columns = ["WIDINYR", "DIVINYR"])

In [0]:
df.MORTGAGE = df.MORTGAGE.map({0:0, 1:1, 3:2, 4:2})

In [0]:
df = df.drop(columns = ["COUNTYFIP", "CITY"])

In [0]:
df['PCT_HHINC'] = df.INCTOT/df.HHINCOME*100 # created new variable

In [0]:
df['PCT_MTG_INC'] = df.MORTAMT1/ (df.HHINCOME/12) * 100 # created new variable

In [0]:
df = df.drop(columns = ["BPL"])

In [0]:
def occupation(x):
  if x == 0:
    return np.nan
  elif x > 0 and x <= 960:
    return 'mgmt, biz, fin'
  elif x > 1000 and x <= 1980:
    return 'comp, eng, sci'
  elif x > 2000 and x <= 2920:
    return 'educ, legal, comm srvs, arts, media'
  elif x >= 3000 and x <= 3550:
    return 'healthcare'
  elif x > 2600 and x <= 4655:
    return 'service'
  elif x >= 4700 and x <= 4965:
    return 'sales and related'
  elif x >= 5000 and x <= 5940:
    return 'office and admin'
  elif x > 6000 and x <= 6130:
    return 'farm, fish, forrestry'
  elif x >= 6200 and x <=6950:
    return 'constr, extract'
  elif x >= 7000 and x<= 7640:
    return 'instal, maint, repair'
  elif x >= 7700 and x <= 8990:
    return 'production'
  elif x > 9000 and x <= 9920:
    return 'transpo and material moving'
  else:
    return 'other' 

In [0]:
df['OCC_BROAD'] = df['OCC'].apply(occupation)

In [49]:
df.OCC_BROAD.value_counts()

mgmt, biz, fin                         237823
service                                186277
educ, legal, comm srvs, arts, media    162155
office and admin                       160034
sales and related                      123320
transpo and material moving             99147
healthcare                              88848
comp, eng, sci                          79089
production                              74316
constr, extract                         64396
instal, maint, repair                   41199
farm, fish, forrestry                    9639
Name: OCC_BROAD, dtype: int64

In [0]:
def ancestry(x):
  if x == 0:
    return np.nan
  elif x > 0 and x <= 99:
    return 'western european'
  elif x >= 100 and x <= 179:
    return 'eatern european'
  elif x >= 181 and x <= 195:
    return 'european, general'
  elif x >= 200 and x <= 296:
    return 'hispanic'
  elif x >= 300 and x <= 337:
    return 'west indies'
  elif x >= 360 and x <= 380:
    return 'non-hispanic south and central amer'
  elif x >= 400 and x <= 496:
    return 'north africa and southwest asia'
  elif x >= 500 and x <= 599:
    return 'subsaharan africa'
  elif x >= 600 and x <=695:
    return 'south asia'
  elif x >= 700 and x <= 796:
    return 'other asia'
  elif x >= 800 and x <= 870:
    return 'pacific'
  elif x > 900 and x <= 994:
    return 'north american'
  else:
    return 'other' 

In [0]:
df['ANCESTR1'] = df['ANCESTR1'].apply(ancestry)

In [0]:
df.to_csv('ipums.csv', index=False)


In [55]:
df.columns.tolist()

['MIGRATE1',
 'VETSTAT',
 'UHRSWORK',
 'OCC',
 'EMPSTAT',
 'YRMARR',
 'ANCESTR1',
 'CLASSWKR',
 'EDUC',
 'INCTOT',
 'RACE',
 'HISPAN',
 'DEGFIELD',
 'AGE',
 'BIRTHYR',
 'MORTGAGE',
 'SEX',
 'PERWT',
 'STRATA',
 'CLUSTER',
 'MARRNO',
 'NCHILD',
 'STATEFIP',
 'MARST',
 'NCHLT5',
 'VALUEH',
 'HHINCOME',
 'MORTAMT1',
 'YEAR',
 'AGEMARR',
 'PCT_HHINC',
 'PCT_MTG_INC',
 'OCC_BROAD']

In [0]:
# Numerical/continuous variables:
# UHRSWORK, AGEMARR, PCT_HHINC, PCT_MTG_INC, MORTGAGE, HHINCOME, INCTOT, MORTAMT1, AGE, 

# Drop from analysis?
# YRMARR, BIRTHYR, YEAR, OCC, PERWT, STRATA, CLUSTER

# Target
# MARST

# Categorical variables to one hot encode:
# MIGRATE1, VETSTAT, EMPSTAT, ANCESTR1, STATEFIP, OCC_BROAD, CLASSWKR, NCHILD, MARRNO, NCHLT5, SEX, DEGFIELD, HISPAN, RACE, EDUC, 